# Individual-based simulation of speciation

This code consists of three classes: RNASeq, Population, and TwoPops. Population class represnts a population as a collection of RNASeq objects. 

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline  

In [29]:
run RNA_pop_v6.py

#### Reference sequence, i.e., the sequence used to deem any sequence viable or inviable:

In [30]:
ref_seq = 'UAGACUAUAAAGUACCGGGAUCUAUUGAAUGUUGGUACCCCUUUCGAGUGAUUGCGCCCCAGUCCGUGAACUAGGAUGCAUAUUUACCUAUUCAGAGGAU'

In [31]:
pop = Population(ref_seq, pop_size=10, mut_rate=1e-1, rec_rate=0.1)

In [32]:
RNASeq.get_hamdist(pop.ancestor.seq, pop.ref_seq.seq)

62

In [33]:
[RNASeq.get_hamdist(pop.ref_seq.seq, RNASeq.convertor(i, inv=True)) for i in pop.population]

[62, 62, 62, 62, 62, 62, 62, 62, 62, 62]

In [34]:
evol = TwoPops(pop)

In [35]:
evol.evolve(50, step=10, verbose=True)

10 10 10
20 10 10
30 10 10
40 10 10
50 10 10


In [41]:
evol.stats['rec_load']

[10.0, 10.0, 10.0, 10.0, 10.0, 10.0]

In [ ]:
import multiprocessing as mp

In [ ]:
def evolve(ref, N, u, r, bp, m, gen):
    pop = Population(ref, pop_size=N, mut_rate=u, rec_rate=r, alpha=bp)
    evo = TwoPops(pop, mig_rate=m)
    evo.evolve(gen, step=25)
    evo.save_stats('/Users/Amfortas/Desktop/')

In [ ]:
def main(N=100, u=1e-3, r=0.1, bp=12, m=0.0, gen=100):
    num_process = 4
    pool = mp.Pool(processes=num_process)
    #results = [pool.apply_async(evolve, args=(ref_seq, N, u, r, bp, m, gen)) for x in np.arange(1, num_process + 1, 1)]
    #results = [pool.apply_async(evolve, args=(ref_seq, N, u, r, bp, m, gen,)) for x in range(num_process)]
    processes = [mp.Process(target=evolve, args=(ref_seq, N, u, r, bp, m, gen,)) for x in range(num_process)]
    # Run processes
    for p in processes:
        p.start()
    # Exit the completed processes
    for p in processes:
        p.join()

In [ ]:
main()

In [ ]:
for i in range(100):
    print np.arange(i, i + 4, 1)

In [ ]:
D = {}
for i in [1e-3, 1e-4, 1e-5]:
    pop = Population(ref_seq, pop_size=50, mut_rate=i, rec_rate=0.)
    evol = TwoPops(pop)
    evol.evolve(2000)
    D[i] = evol.divergence
    print i,

In [ ]:
for i in D.keys():
    print i, D[i]

In [ ]:
sims = {}
count = 0
for i in range(50):
    pop = Population(ref_seq, pop_size=1, mut_rate=1e-4, rec_rate=0.)
    evol = TwoPops(pop)
    evol.evolve(1000)
    sims[count] = evol.stats
    count += 1
    print count,

In [ ]:
file = open("../stats_50_N_1_u_4", 'w')
pickle.dump(sims, file)
file.close()

In [ ]:
D = []
for i in sims.keys():
    D.append(sims[i]['divergence'])
D = np.array(D)

In [ ]:
plt.plot(D.mean(axis=0))

In [ ]:
sims[0]['double_inv']

In [ ]:
single_pop = []
for i in sims.keys():
    single_pop.append([len(j) for j in sims[i]['single']])
single_pop = np.array(single_pop)

double_pop = []
for i in sims.keys():
    double_pop.append([len(j) for j in sims[i]['double']])
double_pop = np.array(double_pop)

triple_pop = []
for i in sims.keys():
    triple_pop.append([len(j) for j in sims[i]['triple']])
triple_pop = np.array(triple_pop)

In [ ]:
x = np.arange(1, 41, 1)
fig = plt.figure(figsize=(10,10))
plt.rcParams.update({'font.family': 'Arial'})
plt.rcParams.update({'font.size': 40})
ax = fig.add_subplot(111)
ax.spines['right'].set_visible(True)
ax.spines['top'].set_visible(True)
ax.spines['bottom'].set_visible(True)
ax.spines['left'].set_visible(True)
ax.spines['left'].set_linewidth(3)
ax.spines['bottom'].set_linewidth(3)
ax.spines['right'].set_linewidth(3)
ax.spines['top'].set_linewidth(3)
ax.spines['bottom'].set_color('black')
ax.spines['left'].set_color('black')
#ax.spines['left'].set_position(('outward', 15))
#ax.spines['bottom'].set_position(('outward', 15))

plt.plot(np.mean(single_pop, axis=0), lw=5, label='single')

plt.plot(np.mean(double_pop, axis=0), lw=5, label='double')

plt.plot(np.mean(triple_pop, axis=0), lw=5, label='triple')

ax.grid(True)

plt.tick_params(axis='x', which='both', bottom='on', top='off',
    labelbottom='on', size=20, width=3, direction='out', pad=25, colors='black')
plt.tick_params(axis='y', which='both', left='on', right='off',
    labelbottom='on', size=20, width=3, direction='out', pad=25, colors='black')
plt.ylabel(r'DMIs', fontsize= 50, labelpad=20)
plt.xlabel(r'Generation ($\times 10^{3}$)', fontsize= 50, labelpad=20)

plt.title(r'$N=1, u=10^{-5}, r=0, m=0$', fontsize= 30)
ax.xaxis.label.set_color('black')

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles,labels, loc='best', fontsize='medium', \
          handlelength=0.5, handletextpad=0.4, framealpha=0, labelspacing=0.1,\
          numpoints=1, markerscale=0.3)
#plt.text(3.2,11.7, r'q', fontsize= 50, style='italic')
#plt.xticks([0, 5, 10, 15, 20],[0, r'$2.5$', r'$5$', r'$7.5$', r'$10$'])
#plt.ylim(0, 20)
#plt.xlim(0, 10)
plt.show()
#plt.savefig('/Users/Amfortas/Desktop/cluster/data/figs/DMI_n1000_u1e-5_r1e-5.png', bbox_inches='tight')

In [ ]:
y = ['a', 'b', 'c']

In [ ]:
[rnd.choice(y) for i in range(10)]

In [ ]:
pop = Population(ref_seq, pop_size=10, mut_rate=1e-4, rec_rate=0.)

In [ ]:
evol = TwoPops(pop)

In [ ]:
evol.evolve(2000, verbose=True)

In [ ]:
evol.divergence

In [ ]:
plt.plot(evol.divergence, [len(i) for i in evol.single])

In [ ]:
x = evol.pop1.introgression_assay(evol.pop2)

In [ ]:
evol.pop1.ref_seq.seq

In [ ]:
evol.pop1.ref_seq.get_bp_distance(RNASeq(evol.pop1.wt_seq))

In [ ]:
import uuid

In [ ]:
unique_filename = str(uuid.uuid4())

In [ ]:
unique_filename

In [ ]:
evol.save_stats('/Users/Amfortas/Desktop', 0)

In [ ]:
f = open('/Users/Amfortas/Desktop/stats_0')
data = pickle.load(f)
f.close()

In [ ]:
data.keys()

In [ ]:
plt.plot(data['divergence'])
plt.plot([len(i) for i in data['single']])
plt.plot([len(i) for i in data['double']])
plt.plot([len(i) for i in data['triple']])
#plt.plot(data['double'])

In [ ]:
x = np.arange(1, 41, 1)
fig = plt.figure(figsize=(10,10))
plt.rcParams.update({'font.family': 'Arial'})
plt.rcParams.update({'font.size': 40})
ax = fig.add_subplot(111)
ax.spines['right'].set_visible(True)
ax.spines['top'].set_visible(True)
ax.spines['bottom'].set_visible(True)
ax.spines['left'].set_visible(True)
ax.spines['left'].set_linewidth(3)
ax.spines['bottom'].set_linewidth(3)
ax.spines['right'].set_linewidth(3)
ax.spines['top'].set_linewidth(3)
ax.spines['bottom'].set_color('black')
ax.spines['left'].set_color('black')
#ax.spines['left'].set_position(('outward', 15))
#ax.spines['bottom'].set_position(('outward', 15))

plt.plot(data['D'], lw=4)


plt.tick_params(axis='x', which='both', bottom='on', top='off',
    labelbottom='on', size=20, width=3, direction='out', pad=25, colors='black')
plt.tick_params(axis='y', which='both', left='on', right='off',
    labelbottom='on', size=20, width=3, direction='out', pad=25, colors='black')
plt.ylabel(r'$D$', fontsize= 50, labelpad=20)
plt.xlabel(r'Time', fontsize= 50, labelpad=20)

#plt.xticks([0, 25, 50], [0, 0.5, 1])
#plt.yticks([0, 0.5, 1])

ax.xaxis.label.set_color('black')

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles,labels, loc='best', fontsize='medium', \
          handlelength=0.5, handletextpad=0.4, framealpha=0, labelspacing=0.1,\
          numpoints=1, markerscale=0.3)
#plt.text(3.2,11.7, r'q', fontsize= 50, style='italic')
#plt.ylim(0, 12);
plt.show()